In [4]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [10]:
# Konfigurasi database tujuan
#source_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
#source_app = firebase_admin.initialize_app(source_cred, {
 #   'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
#})

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [13]:
# Input readable date
start_readable_date = "09-10-2025 04:53:00"  # Format: DD-MM-YYYY HH:MM:SS
end_readable_date = "10-10-2025 17:56:52"  # Format: DD-MM-YYYY HH:MM:SS

# Convert readable date to Unix timestamp
start_timestamp = int(datetime.strptime(start_readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(start_timestamp)
# end_timestamp = int(datetime.now().timestamp())  # Current time in Unix timestamp
end_timestamp = int(datetime.strptime(end_readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(end_timestamp)

1759985580
1760119012


In [16]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-03/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di databse sumber.")

                  dew  humidity  pressure  temperature    timestamp  volt  \
-2085749716  25.28303     64.76   1009.83        32.79  -2085749716  4.15   
-2058618016  24.26897     85.59   1010.58        26.89  -2058618016  4.13   
-2058013216  24.58622     79.03   1010.14        28.58  -2058013216  4.14   
-2055738496  24.92759     95.85   1012.44        25.64  -2055738496  4.09   
-2048394496  25.53694     65.56   1009.82        32.84  -2048394496  4.16   

             humi  pres  temp  
-2085749716   NaN   NaN   NaN  
-2058618016   NaN   NaN   NaN  
-2058013216   NaN   NaN   NaN  
-2055738496   NaN   NaN   NaN  
-2048394496   NaN   NaN   NaN  


In [19]:
df.head(10)

,dew,humidity,pressure,temperature,timestamp,volt,humi,pres,temp
-2085749716,25.28303,64.76,1009.83,32.79,-2085749716,4.15,NaN,NaN,NaN
-2058618016,24.26897,85.59,1010.58,26.89,-2058618016,4.13,NaN,NaN,NaN
-2058013216,24.58622,79.03,1010.14,28.58,-2058013216,4.14,NaN,NaN,NaN
-2055738496,24.92759,95.85,1012.44,25.64,-2055738496,4.09,NaN,NaN,NaN
-2048394496,25.53694,65.56,1009.82,32.84,-2048394496,4.16,NaN,NaN,NaN
-1981025792,24.48321,75.25,1012.11,29.32,-1981025792,4.15,NaN,NaN,NaN
-1981025432,24.21249,74.90,1012.19,29.12,-1981025432,4.14,NaN,NaN,NaN
-1981025312,24.77171,64.14,1011.87,32.42,-1981025312,4.15,NaN,NaN,NaN
-1980881072,24.64882,65.72,1012.17,31.86,-1980881072,4.15,NaN,NaN,NaN
-1980881012,25.70190,67.60,1009.90,32.47,-1980881012,4.15,NaN,NaN,NaN


In [49]:
# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

In [28]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [31]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-03/data', app=dest_app)

In [52]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Data berhasil dipindahkan ke database tujuan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>